In [1]:
import pandas as pd

In [2]:
import json

In [3]:
df = pd.read_csv("Blocks Data - DD_sample_data.csv")

In [4]:
#convert extras from string to dictionary
df['extras'] = df['extras'].apply(lambda x: json.loads(x))

In [5]:
#extract dictionary into df with separate columns for each key
df_extras = df.extras.apply(pd.Series)

In [6]:
df_extras.isna().sum()

name                          0
felon                        51
birthplace                 1636
identification_two            0
race and ethnicity         1578
gender__FL2 DS DE 39       1550
Mentally Incapacitated     1536
birth_location             1104
race__FL4 DS DE 39 v2       927
gender__FL4 DS DE 39 v2     916
former_name                2066
armed_forces               2049
dtype: int64

In [7]:
#merge original df with dictionary df
df_clean = pd.merge(right=df, left=df_extras, right_index = True, left_index=True)

In [8]:
#create csv
df_clean.to_csv("clean_blocks.csv", index=False)